Some details found here:

http://elasticsearch-py.readthedocs.io/en/master/

In [2]:

from pyelasticsearch.client import ElasticSearch

In [3]:
endpoint = 'https://search-test-vu2fwve5ykzm5tsjkut5q4idum.us-east-1.es.amazonaws.com'

In [4]:
es = ElasticSearch(endpoint, port=443)

In [5]:
es.index('contacts',
          'person',
          {'name': 'Jing', 'age': 25, 'title': 'QA Master'},
           id=6)

{'_id': '6',
 '_index': 'contacts',
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'person',
 '_version': 2,
 'created': False,
 'result': 'updated'}

In [6]:
docs = [{'id': 2, 'name': 'Jessica Coder', 'age': 32, 'title': 'Programmer'},
          {'id': 3, 'name': 'Freddy Tester', 'age': 29, 'title': 'Office Assistant'}]
es.bulk((es.index_op(doc, id=doc.pop('id')) for doc in docs),
          index='contacts',
          doc_type='person')

{'errors': False,
 'items': [{'index': {'_id': '2',
    '_index': 'contacts',
    '_shards': {'failed': 0, 'successful': 1, 'total': 2},
    '_type': 'person',
    '_version': 1,
    'created': True,
    'result': 'created',
    'status': 201}},
  {'index': {'_id': '3',
    '_index': 'contacts',
    '_shards': {'failed': 0, 'successful': 1, 'total': 2},
    '_type': 'person',
    '_version': 1,
    'created': True,
    'result': 'created',
    'status': 201}}],
 'took': 17}

In [16]:
es.refresh('contacts')

{'_shards': {'failed': 0, 'successful': 5, 'total': 10}}

In [17]:
es.get('contacts', 'person', 3)

{'_id': '3',
 '_index': 'contacts',
 '_source': {'age': 29, 'name': 'Freddy Tester', 'title': 'Office Assistant'},
 '_type': 'person',
 '_version': 3,
 'found': True}

In [19]:
es.search('name:joe OR name:freddy', index='contacts')

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '1',
    '_index': 'contacts',
    '_score': 0.25811607,
    '_source': {'age': 25, 'name': 'Joe Tester', 'title': 'QA Master'},
    '_type': 'person'},
   {'_id': '3',
    '_index': 'contacts',
    '_score': 0.25811607,
    '_source': {'age': 29,
     'name': 'Freddy Tester',
     'title': 'Office Assistant'},
    '_type': 'person'}],
  'max_score': 0.25811607,
  'total': 2},
 'timed_out': False,
 'took': 2}

In [34]:
es.search(index='search_v1', query={"query": {"range": {"publish_date":{"gte": "2018-02-23",
                                                                       "lte": "2018-02-28"}}},
                                   })
                                                       

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '346',
    '_index': 'search_v1',
    '_score': 1.0,
    '_source': {'abstract': 'In a previous episode, we discussed Markov Decision Processes or MDPs, a framework for decision making and planning.  This episode explores the generalization Partially Observable MDPs (POMDPs) which are an incredibly general framework that describes most every agent based system.',
     'author': 'Kyle',
     'blog_id': 346,
     'content': '<h2>Optimal Decision Making with POMDPs</h2>\n<p>In a previous episode, we discussed <a href="https://dataskeptic.com/blog/episodes/2018/markov-decision-processes">Markov Decision Processes</a> or MDPs.  An MDP is defined by four parameters in the tuple <img className=\'latex-svg\' src=\'http://s3.amazonaws.com/dataskeptic.com/latex/%5C%5Cbig+%5C%5Clangle+S%2C+A%2C+T%2C+R+%5C%5Cbig+%5C%5Crangle.svg\' alt=\'\\big \\langle S, A, T, R \\big \\rangle\' />.  These variables are the states, 

In [12]:
#es.search(index='search_v1', query={"query": {"match": {"title": "AI"}}})

In [13]:
query = {
     'query': {
         'filtered': {
             'query': {
                 'query_string': {'query': 'name:tester'}
             },
             'filter': {
                 'range': {
                     'age': {
                         'from': 27,
                         'to': 37,
                     },
                 },
             },
         },
     },
}
es.search(index='contacts', query=query)

GET /contacts/_search [status:400 request:0.389s]


ElasticHttpError: (400, 'parsing_exception')

In [107]:
es.delete_index('candidates_cv')

{'acknowledged': True}

In [ ]:
import elasticsearch

host = 'YOURHOST.us-east-1.es.amazonaws.com'
awsauth = AWS4Auth(your_access_key, your_secret_key, region, 'es')

es = elasticsearch.Elasticsearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=elasticsearch.connection.RequestsHttpConnection
)
print(es.cluster.health())

In [34]:
from elasticsearch import Elasticsearch
import json
import requests
from pyelasticsearch.client import ElasticSearch

search_v1={ 
    'mapping':{
        'blog':{
            'properties':{
                'blog_id':{'type':'integer'},
                'title':{'type':'string','analyzer':'english'},
                'author':{'type':'string','index':'not_analyzed'},
                'abstract':{'type':'string','analyzer':'english'},
                'date_created':{'type':'date'},
                'publish_date':{'type':'date'},
                'content':{'type':'string','analyzer':'english'}
            }
        }
    }
}



if __name__ == '__main__':
    config = json.load(open('../config/configsql.json', 'r'))
    # TODO: Jing to populate some code from the notebook
    user = config['dev']['mysql']['user']
    password = config['dev']['mysql']['password']
    host = config['dev']['mysql']['host']
    port = config['dev']['mysql']['port']
    dbname = config['dev']['mysql']['dbname']
    aws_access_key_id=config['dev']['aws']['key']
    aws_secret_access_key=config['dev']['aws']['secret']
    bucketname=config['dev']['bucketname']['bucketname']
    endpoint = 'https://search-test-vu2fwve5ykzm5tsjkut5q4idum.us-east-1.es.amazonaws.com'
    client = Elasticsearch(endpoint, port=443)
    client.indices.create(index='search_v1')
   

/usr/local/lib/python3.6/site-packages/elasticsearch/connection/http_urllib3.py:54: UserWarning: Connecting to search-test-vu2fwve5ykzm5tsjkut5q4idum.us-east-1.es.amazonaws.com using SSL with verify_certs=False is insecure.
  'Connecting to %s using SSL with verify_certs=False is insecure.' % host)
/usr/local/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [31]:
client.indices.delete(index='search_v1')

/usr/local/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'acknowledged': True}

In [ ]:
successful_update = False
#result = elastic_search_conn.delete(index=index_name,doc_type='blog',body='{"query":{"match_all":{}}}')
doc = {
    'blog_id'      : int(row['blog_id']),
    'title'        : row['title'],
    'author'       : row['author'],
    'abstract'     : row['abstract'],
    'date_created' : row['date_created'],
    'publish_date' : row['publish_date'],
    'content'      : html
    }
print("Going to update " + row['title'])
result2 = elastic_search_conn.create(index='search_v1',doc_type='blog',body=doc) 

In [6]:
es.get('search_v1', 'blog', 3)

{'_id': '3',
 '_index': 'search_v1',
 '_source': {'abstract': 'In this episode we discuss the techniques and approaches to perform advertising attribution - the process of helping advertisers understand the impact all aspects of their marketing efforts contribute to their overall g',
  'author': 'Kyle',
  'blog_id': 3,
  'content': '<h2>Advertising Attribution with Nathan Janos</h2>\n<p>In this episode we discuss the techniques and approaches to perform advertising attribution - the process of helping advertisers understand the impact all aspects of their marketing efforts contribute to their overall gain.</p>\n<h3>Links to things mentioned during the show:</h3>\n<p><a href="http://longnow.org/">The Long Now Foundation</a></p>\n<p><a href="http://jeffjag.com/">Jeff Jagunich, artist</a></p>\n<p><a href="http://www.convertro.com/">Convertro</a></p>',
  'date_created': '2014-06-06T00:00:00',
  'prettyname': '/episodes/2014/advertising-attribution-with-nathan-janos',
  'publish_date': '201

In [7]:
es.get('search_v1', 'blog', 1)

{'_id': '1',
 '_index': 'search_v1',
 '_source': {'abstract': 'The Data Skeptic Podcast features conversations with topics related to data science, statistics, machine learning, artificial intelligence and the like, all from the perspective of applying critical thinking and the scientific method to evaluate the veracity of claims and efficacy of approac',
  'author': 'Kyle',
  'blog_id': 1,
  'content': '<h2>Introduction</h2>\n<p>The Data Skeptic Podcast features conversations with topics related to data science, statistics, machine learning, artificial intelligence and the like, all from the perspective of applying critical thinking and the scientific method to evaluate the veracity of claims and efficacy of approaches.</p>',
  'date_created': '2014-05-23T00:00:00',
  'prettyname': '/episodes/2014/introduction',
  'publish_date': '2014-05-23T00:00:00',
  'title': 'Introduction'},
 '_type': 'blog',
 '_version': 20,
 'found': True}

In [39]:
es.search(index='search_v1',
					query={
					  "query": {
					    "multi_match": {
					      "query": 'beer',
					      "fields": [
					        "title",
					        "abstract"
					      ]
					    }
					  },
					  "_source": {
					    "includes": [
					      "blog_id",
					      "title",
					      "abstract",
					      "prettyname",
					      "publish_date"
					    ]
					  }
					})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '336',
    '_index': 'search_v1',
    '_score': 3.709467,
    '_source': {'abstract': 'For some interactive clustering, check out the presentation given at the Oktoberfest 2017 RLadies Chicago meetup or the Shiny app built from the same data',
     'blog_id': 336,
     'prettyname': '/projects/2018/beer-in-hand-data-science',
     'publish_date': '2099-01-01T00:00:00',
     'title': 'Beer-in-Hand Data Science'},
    '_type': 'blog'},
   {'_id': '335',
    '_index': 'search_v1',
    '_score': 2.7303836,
    '_source': {'abstract': 'For some interactive clustering, check out the presentation given at the Oktoberfest 2017 RLadies Chicago meetup or the Shiny app built from the same data',
     'blog_id': 335,
     'prettyname': '/projects/2018/beer-in-hand-data-science',
     'publish_date': '2018-02-04T00:00:00',
     'title': 'Beer-in-Hand Data Science'},
    '_type': 'blog'},
   {'_id': '338',
    '_index

In [158]:
#####
es.search(index='github_jobs',
query={
  "query": {
    "bool": {
      "must": [
        {
          "multi_match": {
            "query": "data",
            "fields": [
              "title",
              "description"
            ]
          }
        },
        {
          "multi_match": {
            "query": "San Francisco",
            "fields": "location*"
          }
        }
      ],
      "filter": {
        "range": {
          "created_at": {
            "gte": "2018-03-01"
          }
        }
      }
    }
  }
})


{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '1a847c4e-2334-11e8-9c24-a3dc567273db',
    '_index': 'github_jobs',
    '_score': 3.5378366,
    '_source': {'company': 'dock.io',
     'company_logo': 'http://github-jobs.s3.amazonaws.com/7620801c-2333-11e8-8818-0c005ac94010.png',
     'company_url': 'https://dock.io',
     'created_at': '2018-03-09T00:53:32',
     'description': '<p>We\'re looking for a <strong>Head of Engineering</strong> to lead our growing engineer team who is excited about leading engineering efforts for a system that will transform how user data will be managed for every consumer app across the web.</p>\n\n<p>Someone who is a seasoned professional who has ideally worked in both small scrappy teams, as well as larger structured team. Someone who is capable of solving complex computer science problems and not afraid to roll up their sleeves.</p>\n\n<p><strong>About You</strong>:</p>\n\n<p>Effective team builder: not your first rode

In [151]:
es.search(index='github_jobs',
query={
  "query": {
    "function_score": {
      "query": {
        "multi_match": {
          "query": "data",
          "fields": [
            "title",
            "description"
          ],
          "query": "new york",
          "fields": [
            "location"
          ]
        }
      },
      "functions": [
        {
          "gauss": {
            "created_at": {
              "origin": "now-3d/d",
              "scale": "3d",
              "offset": "3d",
              "decay": 0.3
            }
          }
        }
      ]
    }
  }
})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'ee580f20-1b1b-11e8-830c-5aeaa1587dfd',
    '_index': 'github_jobs',
    '_score': 0.109261476,
    '_source': {'company': 'TapTab (Startup)',
     'company_logo': 'http://github-jobs.s3.amazonaws.com/99325a2e-1b1a-11e8-9c14-844ed9519891.png',
     'company_url': None,
     'created_at': '2018-02-26T17:49:37',
     'description': '<p>TapTab is a cutting-edge, POS-integrated mobile application that lets diners conveniently order and pay for meals, and tip and split tabs, using their mobile phones while in restaurants. And it does much more... </p>\n\n<p>We’re inventing the future of the restaurant ordering and payment experience, and we’re looking for great people to help us create that future.  Right now, we’re seeking a Senior iOS Developer to lead development of our mobile application.  We seek a developer whose code is reliable, comprehensible and scalable, who is product-oriented and detail-obsessed,

In [152]:
es.search(index='github_jobs',
query={
  "query": {
    "function_score": {
      "query": {
        "bool": {
          "must": [
            {
              "multi_match": {
                "query": "data",
                "fields": [
                  "title",
                  "description"
                ]
              }
            },
            {
              "multi_match": {
                "query": "new york",
                "fields": [
                  "location"
                ]
              }
            }
          ]
        }
      },
      "functions": [
        {
          "gauss": {
            "created_at": {
              "origin": "now-3d/d",
              "scale": "3d",
              "offset": "3d",
              "decay": 0.3
            }
          }
        }
      ]
    }
  }
})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '47303ad0-18e9-11e8-98dc-ff4d074efc8d',
    '_index': 'github_jobs',
    '_score': 0.003111516,
    '_source': {'company': 'Sapient Consulting',
     'company_logo': 'http://github-jobs.s3.amazonaws.com/3e419892-18e9-11e8-8489-80909dc9d202.png',
     'company_url': 'http://careers.sapientconsulting.com',
     'created_at': '2018-02-23T22:32:33',
     'description': '<p><strong>Front End Interactive Developer (ReactJS or AngularJS)</strong></p>\n\n<p>At Sapient Global Markets, we are quite literally on the front lines of the biggest issues facing the global markets leaders today – investment banks, asset management firms, wealth managers, oil and energy firms, and government and regulatory agencies.  Driving transformation and leading change within these industries requires big thinkers who bring insight, knowledge, technical acumen and passion for innovation.  We offer the opportunity to join a recognize

In [76]:
es.search(index='github_jobs',
query={
  "query": {
    "function_score": {
      "query": {
        "bool": {
          "must": [
            {
              "multi_match": {
                "query": "data",
                "fields": [
                  "title",
                  "description"
                ]
              }
            },
            {
              "multi_match": {
                "query": "new york",
                "fields": [
                  "location"
                ]
              }
            }
          ],
            "filter": {
          "range": {
              "created_at": {
                "gte": "2018-02-01"
                
            }
            }
            }
        }
      },
      "functions": [
        {
          "gauss": {
            "created_at": {
              "origin": "now-3d/d",
              "scale": "3d",
              "offset": "3d",
              "decay": 0.
            }
          }
        }
      ]
    }
  }
})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '47303ad0-18e9-11e8-98dc-ff4d074efc8d',
    '_index': 'github_jobs',
    '_score': 1.9857442,
    '_source': {'company': 'Sapient Consulting',
     'company_logo': 'http://github-jobs.s3.amazonaws.com/3e419892-18e9-11e8-8489-80909dc9d202.png',
     'company_url': 'http://careers.sapientconsulting.com',
     'created_at': '2018-02-23T22:32:33',
     'description': '<p><strong>Front End Interactive Developer (ReactJS or AngularJS)</strong></p>\n\n<p>At Sapient Global Markets, we are quite literally on the front lines of the biggest issues facing the global markets leaders today – investment banks, asset management firms, wealth managers, oil and energy firms, and government and regulatory agencies.  Driving transformation and leading change within these industries requires big thinkers who bring insight, knowledge, technical acumen and passion for innovation.  We offer the opportunity to join a recognized 

In [91]:
es.search(index='github_jobs',
query={
  "query": {
    "function_score": {
      "query": {
        "bool": {
          "must": [
            {
              "multi_match": {
                "query": "data",
                "fields": [
                  "title",
                  "description"
                ]
              }
            },
            {
              "multi_match": {
                "query": "new york",
                "fields": [
                  "location"
                ]
              }
            }
          ],
            "filter": {
          "range": {
              "created_at": {
                "gte": "2018-02-01"
                
            }
            }
            }
        }
      },
      "functions": [
        {
          "gauss": {
            "created_at": {
              "origin": "now-7d/d",
              "scale": "3d",
              "offset": "15d",
              "decay": 0.1
            }
          }
        }
      ]
    }
  }
})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '2488a75c-0c13-11e8-95d7-29592405f90b',
    '_index': 'github_jobs',
    '_score': 4.0661426,
    '_source': {'company': 'Common Living',
     'company_logo': None,
     'company_url': 'http://Common Living',
     'created_at': '2018-02-07T14:32:41',
     'description': '<p>About Common</p>\n\n<p>Common designs, creates, and operates all-inclusive homes, bringing community, convenience, and flexibility to housing. Common members know their neighbors, have more free time, and the opportunity to transfer to any Common home across the country. Since launching in October 2015, Common has opened more than a dozen homes across New York, San Francisco, Oakland, Washington D.C., and Chicago.</p>\n\n<p>It’s an exciting time to be part of Common’s team. We challenge ourselves every day to not just think about ways to make city living better, but to activate on those ideas in meaningful ways. Our team is comprised 

In [10]:
#es.search(index='candidates_cv', query={"query": {"match": {"name": "Hao Wang"}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_00e50c9c.pdf.txt',
    '_index': 'candidates_cv',
    '_score': 6.0743628,
    '_source': {'email': 'yen-hao@huskers.unl.edu',
     'experience': 'Prudential Life Insurance Company of Taiwan Inc. Taipei, Taiwan\nActuarial Specialist, Valuation Team Mar 2011 ï¿½ Oct 2014\nGenerated monthly statistical reports and quarterly financial projection reports.\nPerformed actuarial experience analysis.\nBuilt US GAAP assumptions for pricing and valuation.\nAnalyzed parameters for premium rate and reserve calculation.\nCompleted US GAAP valuation model and performing the regression testing.\nProvided financial reporting and various actuarial works on US GAAP forecast and valuation.\nCompleted cell testing and loss recognition testing.\nAssisted with IFRS4 and embedded value projects.\nHontai Life Insurance Company Taipei, Tai

In [65]:
es.search(index='candidates_cv', query={"_source" : ["education"],
                                           "query": {"match_all": {}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_B. TolgaOztanResume.pdf.txt',
    '_index': 'candidates_cv',
    '_score': 1.0,
    '_source': {'education': 'UNIVERSITY OF CALIFORNIA IRVINE, IRVINE, CA {C% Gd aC*bGY *CGlY G*d*\n8 F\nUNIVERSITY OF CALIFORNIA IRVINE, IRVINE, CA a Gd GG\n8 F\nGRINNELL COLLEGE, GRINNELL, IA Gd aC*bG\nc : F'},
    '_type': 'candidate'},
   {'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_MarkoBokulic_resume.pdf.txt',
    '_index': 'candidates_cv',
    '_score': 1.0,
    '_source': {},
    '_type': 'candidate'},
   {'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_CV.pdf.txt',
    '_index': 'candidates_cv',
    '_score': 1.0,
    '_source': {'education': 'Postdoctoral Massachusetts Institute of Technology (MIT), USA, 2014 Present.\nPh.D.

In [38]:
es.search(index='candidates_cv', query={"_source" : ["name","education"],
                                           "query": {"term": {"education":"university"}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Jason_Kaszpurenko.pdf.txt',
    '_index': 'candidates_cv',
    '_score': 0.17641081,
    '_source': {'education': 'Ph.D. in Physics\nUniversity of California at Davis - Davis, CA\nDecember 2013\nM.S. in Physics\nUniversity of California at Davis - Davis, CA\nDecember 2009\nB.S. in Physics\nClarkson University - Potsdam, NY\nMay 2007\nB.S. in Electrical Engineering\nClarkson University - Potsdam, NY\nMay 2007',
     'name': 'Jason Kaszpurenko'},
    '_type': 'candidate'},
   {'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_ResumeJasonKaszpurenko.pdf.txt',
    '_index': 'candidates_cv',
    '_score': 0.17641081,
    '_source': {'education': 'Ph.D. in Physics\nUniversity of California at Davis - Davis, CA\nDecember 2013\nM.S. in Physics\nUniversity of California at Dav

In [39]:
es.search(index='candidates_cv', query={"_source" : ["name","education"],
                                           "query": {"term": {"education":"college"}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_afb1de38.pdf.txt',
    '_index': 'candidates_cv',
    '_score': 3.1320229,
    '_source': {'education': 'High School( John Muir Charter)\n.High School Diploma\nCollege(Los Angeles ORT College)\n.AAS in Digital Media 3.37 G.P.A',
     'name': 'Ahmir A'},
    '_type': 'candidate'},
   {'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Lanhao_Chen.pdf.txt',
    '_index': 'candidates_cv',
    '_score': 3.0864248,
    '_source': {'education': 'Master of Science in Business Analytics\nTHE GEORGE WASHINGTON UNIVERSITY, School of Business - Washington, DC\n2014 to 2015\nBachelor of Science in Agricultural Economics\nUNIVERSITY OF MARYLAND, College of Agricultural and Natural Resources - College Park, MD\nSeptember 2012 to May 2014\nBachelor of Science in Rural Development Stu

In [63]:
# phd
es.search(index='candidates_cv',
                    query={
                      "query": {
                        "bool": {
                          "must": {
                            "match": {
                              "education": "phd"
                            }
                          },
                          "filter": {
                            "term": {
                              "education": "university"
                            }
                          }
                        }
                      }
                    })
                   

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_ResumeShuaiZheng.pdf.txt',
    '_index': 'candidates_cv',
    '_score': 4.16966,
    '_source': {'awards': "2013-2015 STEM PhD Fellowship, University of Texas at Arlington\n2011 KAUST Academic Excellence Award (6000USD), KAUST\n2009 KAUST Graduate Fellowship, KAUST\n2008 KAUST Discovery Scholarship, KAUST\n2007 First Prize in National Mathematics Modeling Contest, Jilin University\n2006-2009 National Endeavor Scholarship, Jilin University\n2006-2009 Dean's List, Jilin University\n2006-2009 Elite membership in National Computational Science Undergraduate Project,\nJilin University",
     'education': '1/2013-6/2016 PhD in Computer Science, University of Texas, Arlington, TX.\nGPA: 4.0/4.0. Expected to graduate in 6/2016.\n9/2009-12/2012 MSc (PhD track) in Computer Science, King Abdullah University of Science and',
 

In [61]:
# phd
es.search(index='candidates_cv',
                    query={
                      "query": {
                        "bool": {
                          "must": {
                            "match": {
                              "education": "master"
                            }
                          },
                           "must_not":{
                               "match": {
                              "education": "phd"
                            }
                           },
                          "filter": {
                            "term": {
                              "education": "university"
                            }
                          }
                        }
                      }
                    })

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_resume_MamtaKanvinde.pdf.txt',
    '_index': 'candidates_cv',
    '_score': 2.0461016,
    '_source': {'education': 'ILLINOIS INSTITUTE OF TECHNOLOGY; Chicago, IL\n- Master of Science in Data Science\nUNIVERSITY OF MUMBAI; Mumbai, India\n- Master of Science in Computer Science\nUNIVERSITY OF MUMBAI; Mumbai, India\n- Bachelor of Science in Computer Science',
     'email': 'mamta.kanvinde@gmail.com',
     'name': 'Mamta R',
     'phone': '(404)630-7849',
     'profiles': 'www.linkedin.com/in/mamtakanvinde',
     'projects': 'Generating recommendations by analyzing sentiments of Instagram posts (M.S.) Oct-Dec 2015\n- Executed data collection scripts using Instagram web API in python and collected 1 gb of data\n- Implemented Natural Language Processing techniques such as sentiment analysis to determine features.\n- Per

In [ ]:
es.search(index='candidates_cv', query={"_source" : ["name","education"],
                                           "query": {"bool": {"must_not": {"education":"college"}}}})

In [ ]:
es.search(index='candidates_cv', query={"_source" : ["name","education"],
                                           "query":{"filter": {"not": {"term" :{"education":"master"}}}}})

In [126]:
es.search(index='candidates_cv', query={"_source" : ["education"],
                                           "query": {"match_all": {}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_B. TolgaOztanResume.pdf.txt',
    '_index': 'candidates_cv',
    '_score': 1.0,
    '_source': {'education': 'UNIVERSITY OF CALIFORNIA IRVINE, IRVINE, CA {C% Gd aC*bGY *CGlY G*d*\n8 F\nUNIVERSITY OF CALIFORNIA IRVINE, IRVINE, CA a Gd GG\n8 F\nGRINNELL COLLEGE, GRINNELL, IA Gd aC*bG\nc : F'},
    '_type': 'candidate'},
   {'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_MarkoBokulic_resume.pdf.txt',
    '_index': 'candidates_cv',
    '_score': 1.0,
    '_source': {},
    '_type': 'candidate'},
   {'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_CV.pdf.txt',
    '_index': 'candidates_cv',
    '_score': 1.0,
    '_source': {'education': 'Postdoctoral Massachusetts Institute of Technology (MIT), USA, 2014 Present.\nPh.D.

In [76]:
d_education['hits']['total']

622

In [122]:
len(aa['hits']['hits'])

10

In [125]:
range(len(aa['hits']['hits']))

range(0, 10)

In [124]:
for email in range(len(aa['hits']['hits'])):
    email = aa['hits']['hits'][i]['_source']

TypeError: list indices must be integers or slices, not str

In [111]:
aa['hits']['hits']

[{'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_B. TolgaOztanResume.pdf.txt',
  '_index': 'candidates_cv',
  '_score': 1.0,
  '_source': {},
  '_type': 'candidate'},
 {'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_MarkoBokulic_resume.pdf.txt',
  '_index': 'candidates_cv',
  '_score': 1.0,
  '_source': {'email': 'bokmarko@gmail.com'},
  '_type': 'candidate'},
 {'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_CV.pdf.txt',
  '_index': 'candidates_cv',
  '_score': 1.0,
  '_source': {'email': 'o@mit.edu'},
  '_type': 'candidate'},
 {'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Prasuna_Duggirela_Resume.pdf.txt',
  '_index': 'candidates_cv',
  '_score': 1.0,
  '_source': {'email': 'duggirela@mail.usf.edu'},
  '_type': 'candidate'},
 {'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com

In [99]:
aa = es.search(index='candidates_cv', query={"_source" : ["email"],
                                           "query": {"match_all": {}}})

In [80]:
es.search(index='candidates_cv', query={"query": {"match_all": {}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_B. TolgaOztanResume.pdf.txt',
    '_index': 'candidates_cv',
    '_score': 1.0,
    '_source': {'education': 'UNIVERSITY OF CALIFORNIA IRVINE, IRVINE, CA {C% Gd aC*bGY *CGlY G*d*\n8 F\nUNIVERSITY OF CALIFORNIA IRVINE, IRVINE, CA a Gd GG\n8 F\nGRINNELL COLLEGE, GRINNELL, IA Gd aC*bG\nc : F',
     'experience': 'INSIGHT DATA SCIENCE 8*YYl\nI { Y c c\nDesigned and developed a web-app (www.polytweet.com) to predict political party allegiances from twitter content.\nCollected twitter data from the twitter API, cleaned data and created features using Natural Language Processing methods\nsuch as tokenization, stop-word removal and n-grams and fit a Logistic Regression model with L1 regularization.\nDesigned a front-end web application using flask and bootstrap.\nUNIVERSITY OF CALIFORNIA IRVINE :&* **C*\nI F\nConducted a c

In [127]:
es.search(index='candidates_cv1', query={"_source" : ["education"],
                                           "query": {"match_all": {}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_2-13-16 resume.pdf.txt',
    '_index': 'candidates_cv1',
    '_score': 1.0,
    '_source': {'education': "Columbia University Graduate School of Arts and Sciences Expected May 2016\nNew York, New York\nMS in Quantitative Methods in Social Sciences\nData Science Concentration [6 students]\nCornell University School of Arts and Sciences Aug 2009 Dec 2012\nIthaca, New York\nBA in Economics and Statistical Sciences Double Major [GPA: 3.8]\nGraduated magna cum laude\nRelevant Coursework: Data Mining, Algorithms, Advanced GIS and Spatial Analysis, Exploratory Data Analysis and\nVisualization, Design and Analysis of Sample Survey, Advanced Analytic Techniques, Microeconomics of Government,\nFinance, Econometrics, Statistical Sampling, Economics of Export Led Development, Statistics and Probability,\nMicroeconomics, Microe

In [128]:
es.search(index='candidates_cv', query={"_source" : ["education"],
                                           "query": {"match_all": {}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_B. TolgaOztanResume.pdf.txt',
    '_index': 'candidates_cv',
    '_score': 1.0,
    '_source': {'education': 'UNIVERSITY OF CALIFORNIA IRVINE, IRVINE, CA {C% Gd aC*bGY *CGlY G*d*\n8 F\nUNIVERSITY OF CALIFORNIA IRVINE, IRVINE, CA a Gd GG\n8 F\nGRINNELL COLLEGE, GRINNELL, IA Gd aC*bG\nc : F'},
    '_type': 'candidate'},
   {'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_MarkoBokulic_resume.pdf.txt',
    '_index': 'candidates_cv',
    '_score': 1.0,
    '_source': {},
    '_type': 'candidate'},
   {'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_CV.pdf.txt',
    '_index': 'candidates_cv',
    '_score': 1.0,
    '_source': {'education': 'Postdoctoral Massachusetts Institute of Technology (MIT), USA, 2014 Present.\nPh.D.